# Assign positives and negatives 

In [1]:
import json
import threading
import concurrent.futures
import bz2
import csv
import time

import pandas
import py2neo

## Connect to neo4j server

In [2]:
# Override the default py2neo timeout
py2neo.packages.httpstream.http.socket_timeout = 1e8

In [3]:
with open('servers.json') as read_file:
    instances = json.load(read_file)

name_to_neo = dict()
for instance in instances:
    uri = 'http://localhost:{}/db/data/'.format(instance['port'])
    name_to_neo[instance['name']] = py2neo.Graph(uri)

## Read metapaths

In [4]:
with open('data/metapaths.json') as read_file:
    metapaths = json.load(read_file)

metapaths.sort(key=lambda x: x['join_complexities'][0])
len(metapaths)

1206

## Read hetnet-compound-disease pairs

In [5]:
part_df = pandas.read_table('data/partitions.tsv')
part_df['neo'] = part_df.hetnet.map(name_to_neo)
parts = list(part_df.itertuples())
part_df.head(2)

,hetnet,compound_id,disease_id,status,primary,neo
0,rephetio-v2.0_perm-5,DB00014,DOID:0060073,0,1,<Graph uri='http://localhost:7505/db/data/'>
1,rephetio-v2.0_perm-2,DB00014,DOID:1024,0,1,<Graph uri='http://localhost:7502/db/data/'>


## Set up queries

In [6]:
# Total number of queries
total_queries = len(metapaths) * len(part_df)
'{:,}'.format(total_queries)

'46,867,572'

In [7]:
parts = [row for row in part_df.itertuples()]

def generate_parameters(max_elems=None):
    """Generate compound, disease, metapath combinations"""
    n = 0
    for metapath_dict in metapaths:
        metapath = metapath_dict['abbreviation']
        query = metapath_dict['dwpc_query']
        for part_info in parts:
            if max_elems is not None and n == max_elems:
                break
            yield {
                'neo': part_info.neo,
                'hetnet': part_info.hetnet,
                'compound_id': part_info.compound_id,
                'disease_id': part_info.disease_id,
                'metapath': metapath,
                'query': query,
                'w': 0.4,
            }
            n += 1

In [8]:
def compute_dwpc(neo, hetnet, query, metapath, compound_id, disease_id, w):
    """Execute the neo4j query and write results to file"""
    start = time.time()
    results = neo.cypher.execute(query, source=compound_id, target=disease_id, w=w)
    record = results.one
    seconds = '{0:.4g}'.format(time.time() - start)
    row = hetnet, compound_id, disease_id, metapath, record['PC'], w, '{0:.6g}'.format(record['DWPC']), seconds
    with writer_lock:
        writer.writerow(row)

## Execute queries

In [9]:
%%time

# Parameters
workers = 12
max_elems = None

# Prepare writer
path = 'data/dwpc.tsv.bz2'
write_file = bz2.open(path, 'wt')
writer = csv.writer(write_file, delimiter='\t')
writer.writerow(['hetnet', 'compound_id', 'disease_id', 'metapath', 'PC', 'w', 'DWPC', 'seconds'])

# Create ThreadPoolExecutor
executor = concurrent.futures.ThreadPoolExecutor(max_workers=workers)
writer_lock = threading.Lock()

# Submit jobs
n_queries = 0
for params in generate_parameters(max_elems):
    while executor._work_queue.qsize() > 10000:
        print('Submitted queries: {} ({:.4%})'.format(n_queries, n_queries / total_queries), end='\r')
        time.sleep(1)
    executor.submit(compute_dwpc, **params)
    n_queries += 1

# Shutdown and close
executor.shutdown()
write_file.close()

CPU times: user 1d 11h 47min 6s, sys: 3h 33min 58s, total: 1d 15h 21min 5s
Wall time: 4d 11h 53min


In [10]:
n_queries

46867572